<a href="https://colab.research.google.com/github/bijay2051/Bijay_NLP/blob/main/NLP_Assignment_2_Bijay_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment-2: n-gram TF-IDF and document similarity <br>
## Tag: MDS555-2023-Assignment-2 <br>
Objective
- Learn Term Frequency
- Compute document similarity

## Your Details

In [ ]:
import datetime
student_rollno = 11
student_name = 'Bijay Prasad Ghimire'
assignment_tag = 'MDS555-2023-Assignment-2'

In [ ]:
# from checker_utils import done
def done(task):
    _date = datetime.datetime.now()
    task = task + ": " + str(_date)
    print('='*len(task), '\n', task , '\n', '='*len(task), sep='')
    pass

## Tasks <br>
### 1) Task 1:<br>
Dataset Preparation: Prepare the Nepali news dataset (hint: you can
obtain text from news websites, at least 20 different news of 2/3 different
categories). <br>
Host the dataset in the public git repository.
In your notebook data should be downloaded from git or some other public
places. <br>
No additional step should be done to get the notebook working.
You can reuse the dataset of Assignment 1 as well.

In [ ]:
# TODO
import requests
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')


raw_url = f'https://raw.githubusercontent.com/bijay2051/Bijay_NLP/main/news_all.txt'
response = requests.get(raw_url)
if response.status_code == 200:
    file_content = response.text

    print(file_content)
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")

nepali_strings=[file_content]
nepali_strings = ' '.join(nepali_strings)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


सूर्यदर्शन सहकारीका उपाध्यक्ष दर्लामी काठमाडौंबाट पक्राउ
पोखरा — पोखराको सूर्यदर्शन बचत तथा ऋण सहकारी संस्थाका सञ्चालक समिति उपाध्यक्ष कैलाशकुमार दर्लामी बिहीबार काठमाडौंबाट पक्राउ परेका छन् । सहकारी ठगीको फरार सूचीमा रहेका उनी लुकिछिपी बसेको अवस्थामा काठमाडौं प्रहरीले पक्राउ गरेको हो । 
दर्लामीलाई लिन कास्की प्रहरीको टोली धादिङको गजुरीतर्फ गइसकेको छ । दर्लामीलाई बिहीबार राति नै पोखरा ल्याइने कास्की प्रहरी प्रमुख मोहनकुमार थापाले बताए । ‘कास्की प्रहरीको अनुरोधमा काठमाडौंले पक्राउ गरेको हो,’ उनले भने, ‘फरारको सूचीमा भएको बाँकीलाई पक्राउ गर्न गम्भीर छौं ।’
रकम हिनामिनामा रास्वपा अध्यक्ष रवि लामिछाने समेतको संलग्नता देखिएको उक्त सहकारी ठगीको आरोपमा १९ जना विरुद्ध पक्राउ पूर्जी जारी भएको थियो । त्यसमध्ये सञ्चालक समिति कोषाध्यक्ष कुमार रम्तेल यसअघि नै पक्राउ परिसकेका छन् । उनी कास्की कारागारमा पूर्पक्षका लागि थुनामा छन् ।
पूर्वअध्यक्ष गितेन्द्रबाबु राईसहित वर्तमान सञ्चालक समिति, पूर्वसञ्चालक समिति, लेखाका कर्मचारीसहित १७ जना अझै फरार छन् । प्रहरीले राईसहित जिल्ला प्रहरी कार्यालय कास्कीद्वार

In [ ]:
#Tokenizing the string
tokens = word_tokenize(nepali_strings)

#using unicode to create range of regular expression
pattern = re.compile(u'[\u0900-\u097F]').pattern            #For all devenagri unicode
pattern_number = re.compile(u'[\u0966-\u096F]').pattern     #for numeric devnagari
non_devanagaric = re.compile(u'[^\u0900-\u097F]').pattern   #for all non devenagari unicode

#normalizing the tokens to remove numeric, non devanagric characters and punctuations
modified_token = []
for word in tokens:
    if re.search("।",word):
        word = re.sub("।","",word)      #replacing pudabiram "।" in a word with empty character
    if not re.search(non_devanagaric,word) and not re.search(pattern_number,word) and len(word)>0:
        modified_token.append(word)
    if re.search(non_devanagaric,word) and len(word)>0: #working on words with non devanagric characters
        word = re.sub(non_devanagaric," ",word)         #replacing non devanagric characters with space
        word = word_tokenize(word)                      #replacing the characters with space results in two or more words. Thus tokenizing to get separate words
        for sub in word:
            if not re.search(pattern_number,sub) and len(word)>0:
                modified_token.append(sub)

#filtering and removing stopwords from the token
stop_words = set(stopwords.words('nepali'))           #creating set of nepali stop words
filtered_with_stop_words= []
for token in modified_token:
    if token not in stop_words:
      filtered_with_stop_words.append(token)
filtered_with_stop_words

['सूर्यदर्शन',
 'सहकारीका',
 'उपाध्यक्ष',
 'दर्लामी',
 'काठमाडौंबाट',
 'पक्राउ',
 'पोखरा',
 'पोखराको',
 'सूर्यदर्शन',
 'बचत',
 'ऋण',
 'सहकारी',
 'संस्थाका',
 'सञ्चालक',
 'समिति',
 'उपाध्यक्ष',
 'कैलाशकुमार',
 'दर्लामी',
 'बिहीबार',
 'काठमाडौंबाट',
 'पक्राउ',
 'परेका',
 'सहकारी',
 'ठगीको',
 'फरार',
 'सूचीमा',
 'उनी',
 'लुकिछिपी',
 'बसेको',
 'अवस्थामा',
 'काठमाडौं',
 'प्रहरीले',
 'पक्राउ',
 'दर्लामीलाई',
 'लिन',
 'कास्की',
 'प्रहरीको',
 'टोली',
 'धादिङको',
 'गजुरीतर्फ',
 'गइसकेको',
 'दर्लामीलाई',
 'बिहीबार',
 'राति',
 'पोखरा',
 'ल्याइने',
 'कास्की',
 'प्रहरी',
 'प्रमुख',
 'मोहनकुमार',
 'थापाले',
 'बताए',
 'कास्की',
 'प्रहरीको',
 'अनुरोधमा',
 'काठमाडौंले',
 'पक्राउ',
 'फरारको',
 'सूचीमा',
 'बाँकीलाई',
 'पक्राउ',
 'गम्भीर',
 'रकम',
 'हिनामिनामा',
 'रास्वपा',
 'अध्यक्ष',
 'रवि',
 'लामिछाने',
 'समेतको',
 'संलग्नता',
 'देखिएको',
 'उक्त',
 'सहकारी',
 'ठगीको',
 'आरोपमा',
 'जना',
 'विरुद्ध',
 'पक्राउ',
 'पूर्जी',
 'जारी',
 'त्यसमध्ये',
 'सञ्चालक',
 'समिति',
 'कोषाध्यक्ष',
 'कुमार',
 'रम्तेल',
 '

In [ ]:
done('Task 1')

Task 1: 2023-12-28 17:59:55.257330


### 2) Task 2:<br>
Prepare one-gram, bi-gram, tri-gram vocabulary

In [ ]:
# TODO
#creating one-gram, bi-gram and tri-gram vocabulary
one_gram = list(ngrams(filtered_with_stop_words,1))
one_gram_vocab = [" ".join(word) for word in one_gram]    #creating list of one-gram vocabulary

bi_gram = list(ngrams(filtered_with_stop_words,2))
bi_gram_vocab = [" ".join(word) for word in bi_gram]       #creating list of bi-gram vocabulary

tri_gram = list(ngrams(filtered_with_stop_words,3))
tri_gram_vocab = [" ".join(word) for word in tri_gram]     #creating list of tri-gram vocabulary
print(bi_gram_vocab)

['सूर्यदर्शन सहकारीका', 'सहकारीका उपाध्यक्ष', 'उपाध्यक्ष दर्लामी', 'दर्लामी काठमाडौंबाट', 'काठमाडौंबाट पक्राउ', 'पक्राउ पोखरा', 'पोखरा पोखराको', 'पोखराको सूर्यदर्शन', 'सूर्यदर्शन बचत', 'बचत ऋण', 'ऋण सहकारी', 'सहकारी संस्थाका', 'संस्थाका सञ्चालक', 'सञ्चालक समिति', 'समिति उपाध्यक्ष', 'उपाध्यक्ष कैलाशकुमार', 'कैलाशकुमार दर्लामी', 'दर्लामी बिहीबार', 'बिहीबार काठमाडौंबाट', 'काठमाडौंबाट पक्राउ', 'पक्राउ परेका', 'परेका सहकारी', 'सहकारी ठगीको', 'ठगीको फरार', 'फरार सूचीमा', 'सूचीमा उनी', 'उनी लुकिछिपी', 'लुकिछिपी बसेको', 'बसेको अवस्थामा', 'अवस्थामा काठमाडौं', 'काठमाडौं प्रहरीले', 'प्रहरीले पक्राउ', 'पक्राउ दर्लामीलाई', 'दर्लामीलाई लिन', 'लिन कास्की', 'कास्की प्रहरीको', 'प्रहरीको टोली', 'टोली धादिङको', 'धादिङको गजुरीतर्फ', 'गजुरीतर्फ गइसकेको', 'गइसकेको दर्लामीलाई', 'दर्लामीलाई बिहीबार', 'बिहीबार राति', 'राति पोखरा', 'पोखरा ल्याइने', 'ल्याइने कास्की', 'कास्की प्रहरी', 'प्रहरी प्रमुख', 'प्रमुख मोहनकुमार', 'मोहनकुमार थापाले', 'थापाले बताए', 'बताए कास्की', 'कास्की प्रहरीको', 'प्रहरीको अनुरोधमा', 'अन

In [ ]:
done('Task 2')

Task 2: 2023-12-28 18:00:03.392037


### 3) Task 3:<br>
Compute TF-IDF vectors for each vocabulary


In [ ]:
# TODO
#computing TF-IDF vectors for each one-gram, bi-gram and tri-gram vocabulary to find similarity of vocab with entire corpus
Tfidf = TfidfVectorizer()
tfid_one_gram = Tfidf.fit_transform(one_gram_vocab)
tfid_bi_gram = Tfidf.fit_transform(bi_gram_vocab)
tfid_tri_gram = Tfidf.fit_transform(tri_gram_vocab)
print(tfid_bi_gram)

  (0, 983)	0.5504356015679358
  (0, 883)	0.5208707009601075
  (0, 702)	0.652467900672417
  (1, 686)	0.4194089134946258
  (1, 132)	0.6362011072165372
  (1, 983)	0.647567999871604
  (2, 467)	0.6149167637773887
  (2, 686)	0.43404093717509523
  (2, 132)	0.6583964143900756
  (3, 380)	0.6751678683090724
  (3, 467)	0.7376641170634389
  (4, 549)	0.6881559853950467
  (4, 380)	0.7255627745171138
  (5, 241)	0.7293145243862209
  (5, 549)	0.6841785764837278
  (6, 241)	1.0
  (7, 241)	0.48846975169993273
  (7, 883)	0.544393302759196
  (7, 702)	0.681933452460825
  (8, 600)	0.6285350624831225
  (8, 883)	0.48524891984639246
  (8, 702)	0.607846330100994
  (9, 151)	0.6702162355275147
  (9, 600)	0.742165882828985
  (10, 151)	0.7420248219378031
  :	:
  (6882, 829)	1.0
  (6883, 15)	0.6528979357496979
  (6883, 423)	0.757445896083531
  (6884, 602)	0.5802258022370513
  (6884, 15)	0.44361354966085514
  (6884, 779)	0.44908958456491244
  (6884, 423)	0.5146489891284939
  (6885, 602)	0.6064446988973724
  (6885, 582)

In [ ]:
done('Task 3')

Task 3: 2023-12-28 18:00:12.193552


### 4) Task 4:<br>
Compute document similarity matrix (if your document size = N , this
will result in the NxN matrix) for each vocab list.


In [ ]:
# TODO
#Computing similirity matrix between different vocabulary with in corpus
one_gram_cosine = cosine_similarity(tfid_one_gram,tfid_one_gram)
bi_gram_cosine = cosine_similarity(tfid_bi_gram,tfid_bi_gram)
tri_gram_cosine = cosine_similarity(tfid_tri_gram,tfid_tri_gram)
print(one_gram_cosine)
print(bi_gram_cosine)
print(tri_gram_cosine)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[1.         0.35644448 0.         ... 0.         0.         0.        ]
 [0.35644448 1.         0.60091317 ... 0.         0.         0.        ]
 [0.         0.60091317 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.78806745 0.        ]
 [0.         0.         0.         ... 0.78806745 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[[1.         0.60706761 0.36642586 ... 0.         0.         0.        ]
 [0.60706761 1.         0.7216131  ... 0.         0.         0.        ]
 [0.36642586 0.7216131  1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.94239822 0.73841607]
 [0.         0.         0.         ... 0.94239822 1.         0.78354994]
 [0.         0.         0.    

In [ ]:
done('Task 4')

Task 4: 2023-12-28 18:00:16.316878


### 5) Task 5:<br>
Write your interpretation on the result of Task 4.

In task 4, I have computed document similarity matrix using method of cosine similarity for one-gram, bi-gram, tri-gram vocabulary. The symmetric matrix is created between documents and the similarity between two documents is denoted by numeric value. The value of 1 represents the two document are similar to each other in the corpus. While the value of 0 represents two document are less similar. More the value closer to 1 there is more similarity and more value is closer to 0 there is less similarity

In [ ]:
done('Task 5')

Task 5: 2023-12-28 18:00:22.042683
